# Strats+Text Target Task

- binary classification

- physiological features + clinical text

## Hardware check

In [1]:
# gpu check
!nvidia-smi

Sun Jul  9 17:21:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3A:00.0 Off |                    0 |
| N/A   32C    P0    38W / 300W |      9MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# check number of cores
import multiprocessing

cores = multiprocessing.cpu_count() 
cores

80

## Environment Prep

In [3]:
cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis/

/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis


In [4]:
# from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Lambda
# from tensorflow.keras.models import Model
from tensorflow.keras import models
import pickle
import numpy as np
from tqdm import tqdm
tqdm.pandas()
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, EarlyStopping
import pandas as pd
import json
from torch.utils.data import Dataset
from transformers import AutoTokenizer, pipeline, AutoModel
import resources.smart_cond as sc
# from google.colab import files

2023-07-09 17:22:04.236450: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-09 17:22:07.338368: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load Data

In [5]:
# data_path = 'Data/sepsis_removed_0.pkl'
# pkl = pickle.load(open(data_path, 'rb'))
# data = pkl[0]
# oc = pkl[1]
# train_ind = pkl[2]
# valid_ind = pkl[3]
# test_ind = pkl[4]
# del pkl

In [ ]:
# # Filter labeled data in first 24h.
# data = data.loc[data.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]
# data = data.loc[(data.hour>=0)&(data.hour<=24)]
# oc = oc.loc[oc.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]
# # Get y and N.
# y = np.array(oc.sort_values(by='ts_ind')['in_hospital_sepsis']).astype('float32')
# N = data.ts_ind.max() + 1
# # Get static data with mean fill and missingness indicator.
# static_varis = ['Age', 'Gender']
# ii = data.variable.isin(static_varis)
# static_data = data.loc[ii]
# data = data.loc[~ii]
# def inv_list(l, start=0):
#     d = {}
#     for i in range(len(l)):
#         d[l[i]] = i+start
#     return d
# static_var_to_ind = inv_list(static_varis)
# D = len(static_varis)
# demo = np.zeros((N, D))
# for row in tqdm(static_data.itertuples()):
#     demo[row.ts_ind, static_var_to_ind[row.variable]] = row.value
# # Normalize static data.
# means = demo.mean(axis=0, keepdims=True)
# stds = demo.std(axis=0, keepdims=True)
# stds = (stds==0)*1 + (stds!=0)*stds
# demo = (demo-means)/stds
# # Trim to max len.
# data = data.sample(frac=1)
# data = data.groupby('ts_ind').head(880)
# # Get N, V, var_to_ind.
# N = data.ts_ind.max() + 1
# varis = sorted(list(set(data.variable)))
# V = len(varis)
# def inv_list(l, start=0):
#     d = {}
#     for i in range(len(l)):
#         d[l[i]] = i+start
#     return d
# var_to_ind = inv_list(varis, start=1)
# data['vind'] = data.variable.map(var_to_ind)
# data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])
# # Add obs index.
# data = data.sort_values(by=['ts_ind']).reset_index(drop=True)
# data = data.reset_index().rename(columns={'index':'obs_ind'})
# data = data.merge(data.groupby('ts_ind').agg({'obs_ind':'min'}).reset_index().rename(columns={ \
#                                                             'obs_ind':'first_obs_ind'}), on='ts_ind')
# data['obs_ind'] = data['obs_ind'] - data['first_obs_ind']
# # Find max_len.
# max_len = data.obs_ind.max()+1
# print ('max_len', max_len)
# # Generate times_ip and values_ip matrices.
# # times_inp = np.zeros((N, max_len), dtype='float32')
# # values_inp = np.zeros((N, max_len), dtype='float32')
# # varis_inp = np.zeros((N, max_len), dtype='int32')
# texts_inp = np.empty([N, max_len], dtype=object)
# for row in tqdm(data.itertuples()):
#     ts_ind = row.ts_ind
#     l = row.obs_ind
#     # times_inp[ts_ind, l] = row.hour
#     if isinstance(row.value, str):
#         # values_inp[ts_ind, l] = 1.0
#         texts_inp[ts_ind, l] = row.value
#     else:
#         # values_inp[ts_ind, l] = row.value
#         texts_inp[ts_ind, l] = ''
#     # varis_inp[ts_ind, l] = row.vind
    
# data.drop(columns=['obs_ind', 'first_obs_ind'], inplace=True)
# # Generate 3 sets of inputs and outputs.
# # train_ip = [ip[train_ind] for ip in [demo, times_inp, values_inp, varis_inp, texts_inp]]
# # valid_ip = [ip[valid_ind] for ip in [demo, times_inp, values_inp, varis_inp, texts_inp]]
# # test_ip = [ip[test_ind] for ip in [demo, times_inp, values_inp, varis_inp, texts_inp]]
# # del times_inp, values_inp, varis_inp
# train_ip = [ip[train_ind] for ip in [texts_inp]]
# valid_ip = [ip[valid_ind] for ip in [texts_inp]]
# test_ip = [ip[test_ind] for ip in [texts_inp]]
# del texts_inp

In [ ]:
# train_texts = train_ip[0]
# valid_texts = valid_ip[0]
# test_texts = test_ip[0]
# del train_ip, valid_ip, test_ip

In [ ]:
# concat_train_texts = []
# for train_text in tqdm(train_texts):
#     train_text = train_text[train_text != None]
#     train_text = train_text[train_text != '']
#     concat_text = ' '.join(list(train_text))
#     concat_train_texts.append(concat_text)    

In [ ]:
# concat_valid_texts = []
# for valid_text in tqdm(valid_texts):
#     valid_text = valid_text[valid_text != None]
#     valid_text = valid_text[valid_text != '']
#     concat_text = ' '.join(list(valid_text))
#     concat_valid_texts.append(concat_text)    

In [ ]:
# concat_test_texts = []
# for test_text in tqdm(test_texts):
#     test_text = test_text[test_text != None]
#     test_text = test_text[test_text != '']
#     concat_text = ' '.join(list(test_text))
#     concat_test_texts.append(concat_text)    

In [ ]:
# del train_texts, valid_texts, test_texts

### Text Embedding

In [ ]:
# from simpletransformers.language_representation import RepresentationModel
# from simpletransformers.config.model_args import ModelArgs

In [ ]:
# model_args = ModelArgs(max_seq_length=512, silent = False)
# model = RepresentationModel(
#     "bert", "emilyalsentzer/Bio_ClinicalBERT", args=model_args)
# train_text_features = model.encode_sentences(concat_train_texts, combine_strategy="mean")

In [ ]:
# model_args = ModelArgs(max_seq_length=512, silent = False)
# model = RepresentationModel(
#     "bert", "emilyalsentzer/Bio_ClinicalBERT", args=model_args)
# valid_text_features = model.encode_sentences(concat_valid_texts, combine_strategy="mean")

In [ ]:
# model_args = ModelArgs(max_seq_length=512, silent = False)
# model = RepresentationModel(
#     "bert", "emilyalsentzer/Bio_ClinicalBERT", args=model_args)
# test_text_features = model.encode_sentences(concat_test_texts, combine_strategy="mean")

In [ ]:
# # dump to json just in case
# pickle.dump([train_text_features, valid_text_features, test_text_features], open('text_features_target.pkl','wb'))

In [ ]:
# del concat_train_texts, concat_valid_texts, concat_test_texts

In [ ]:
# data_path = 'Data/text_features_target.pkl'
# train_text, valid_text, test_text = pickle.load(open(data_path, 'rb'))

In [ ]:
data_path = 'Data/classification_embs.pkl'
train_text, valid_text, test_text = pickle.load(open(data_path, 'rb'))

In [ ]:
# from keras.utils import pad_sequences
# train_text_features = pad_sequences(train_text, maxlen=33792, padding='post')
# train_text_features.shape

In [ ]:
# valid_text_features = pad_sequences(valid_text, maxlen=33792, padding='post')
# valid_text_features.shape

In [ ]:
# test_text_features = pad_sequences(test_text, maxlen=33792, padding='post')
# test_text_features.shape

### Physio Data

In [ ]:
data_path = 'Data/sepsis_removed_0.pkl'
pkl = pickle.load(open(data_path, 'rb'))
data = pkl[0]
oc = pkl[1]
train_ind = pkl[2]
valid_ind = pkl[3]
test_ind = pkl[4]
del pkl

In [ ]:
data.loc[data['variable'] == 'Text', 'value'] = 1
data

,ts_ind,hour,variable,value,TABLE,mean,std
0,10223,467.816667,Text,1,noteevents,1.000000,1.000000
1,18407,28.016667,Text,1,noteevents,1.000000,1.000000
2,40300,155.166667,Text,1,noteevents,1.000000,1.000000
3,23747,52.383333,Text,1,noteevents,1.000000,1.000000
4,2357,73.133333,Text,1,noteevents,1.000000,1.000000
...,...,...,...,...,...,...,...
82886223,57281,20.400000,MBP,0.195381,chart,78.552377,17.645628
82886224,57281,20.400000,O2 Saturation,-0.678068,chart,96.820961,4.160290
82886225,57281,20.400000,RR,0.179866,chart,26.278501,15.130729
82886226,57281,20.400000,SBP,-0.404061,chart,120.239648,25.341836


In [ ]:
# Filter labeled data in first 24h.
data = data.loc[data.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]
data = data.loc[(data.hour>=0)&(data.hour<=24)]
oc = oc.loc[oc.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]
# Fix age.
data.loc[(data.variable=='Age')&(data.value>200), 'value'] = 91.4
# Get y and N.
y = np.array(oc.sort_values(by='ts_ind')['in_hospital_sepsis']).astype('float32')
N = data.ts_ind.max() + 1
# Get static data with mean fill and missingness indicator.
static_varis = ['Age', 'Gender']
ii = data.variable.isin(static_varis)
static_data = data.loc[ii]
data = data.loc[~ii]
def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d
static_var_to_ind = inv_list(static_varis)
D = len(static_varis)
demo = np.zeros((N, D))
for row in tqdm(static_data.itertuples()):
    demo[row.ts_ind, static_var_to_ind[row.variable]] = row.value
# Normalize static data.
means = demo.mean(axis=0, keepdims=True)
stds = demo.std(axis=0, keepdims=True)
stds = (stds==0)*1 + (stds!=0)*stds
demo = (demo-means)/stds
# Trim to max len.
data = data.sample(frac=1)
data = data.groupby('ts_ind').head(880)
# Get N, V, var_to_ind.
N = data.ts_ind.max() + 1
varis = sorted(list(set(data.variable)))
V = len(varis)
def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d
var_to_ind = inv_list(varis, start=1)
data['vind'] = data.variable.map(var_to_ind)
data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])
# Add obs index.
data = data.sort_values(by=['ts_ind']).reset_index(drop=True)
data = data.reset_index().rename(columns={'index':'obs_ind'})
data = data.merge(data.groupby('ts_ind').agg({'obs_ind':'min'}).reset_index().rename(columns={ \
                                                            'obs_ind':'first_obs_ind'}), on='ts_ind')
data['obs_ind'] = data['obs_ind'] - data['first_obs_ind']
# Find max_len.
max_len = data.obs_ind.max()+1
print ('max_len', max_len)
# Generate times_ip and values_ip matrices.
times_inp = np.zeros((N, max_len), dtype='float32')
values_inp = np.zeros((N, max_len), dtype='float32')
varis_inp = np.zeros((N, max_len), dtype='int32')
for row in tqdm(data.itertuples()):
    ts_ind = row.ts_ind
    l = row.obs_ind
    times_inp[ts_ind, l] = row.hour
    values_inp[ts_ind, l] = row.value
    varis_inp[ts_ind, l] = row.vind
data.drop(columns=['obs_ind', 'first_obs_ind'], inplace=True)
# Generate 3 sets of inputs and outputs.
train_ip = [ip[train_ind] for ip in [demo, times_inp, values_inp, varis_inp]]
valid_ip = [ip[valid_ind] for ip in [demo, times_inp, values_inp, varis_inp]]
test_ip = [ip[test_ind] for ip in [demo, times_inp, values_inp, varis_inp]]
del times_inp, values_inp, varis_inp
# train_op = y[train_ind]
# valid_op = y[valid_ind]
# test_op = y[test_ind]
# del y

114564it [00:00, 763543.49it/s]


max_len 880


19267073it [00:27, 713306.09it/s]


In [ ]:
train_ip.append(train_text)
valid_ip.append(valid_text)
test_ip.append(test_text)

In [ ]:
# data_path = 'Data/sepsis_removed_0.pkl'
pkl = pickle.load(open(data_path, 'rb'))
data = pkl[0]
oc = pkl[1]
train_ind = pkl[2]
valid_ind = pkl[3]
test_ind = pkl[4]
del pkl

In [ ]:
oc = oc.sort_values(by='ts_ind')
oc

,ts_ind,HADM_ID,SUBJECT_ID,in_hospital_sepsis
0,0,110404,268,1
1,1,188028,270,0
2,2,173727,271,0
3,3,164716,272,0
4,4,158689,273,0
...,...,...,...,...
54862,57277,182476,83967,0
29333,57278,118320,23200,0
54573,57279,146497,73807,0
14317,57280,118512,10762,0


In [ ]:
reset_oc = oc.reset_index()
reset_oc

,index,ts_ind,HADM_ID,SUBJECT_ID,in_hospital_sepsis
0,0,0,110404,268,1
1,1,1,188028,270,0
2,2,2,173727,271,0
3,3,3,164716,272,0
4,4,4,158689,273,0
...,...,...,...,...,...
57277,54862,57277,182476,83967,0
57278,29333,57278,118320,23200,0
57279,54573,57279,146497,73807,0
57280,14317,57280,118512,10762,0


In [ ]:
train_op = reset_oc['in_hospital_sepsis'][train_ind]
valid_op = reset_oc['in_hospital_sepsis'][valid_ind]
test_op = reset_oc['in_hospital_sepsis'][test_ind]

In [ ]:
train_op = train_op.to_numpy(dtype='float32')
valid_op = valid_op.to_numpy(dtype='float32')
test_op = test_op.to_numpy(dtype='float32')

In [ ]:
def get_res(y_true, y_pred):
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    pr_auc = auc(recall, precision)
    minrp = np.minimum(precision, recall).max()
    roc_auc = roc_auc_score(y_true, y_pred)
    return [roc_auc, pr_auc, minrp]

######################################################################################################## 
######################################################################################################## 
class_weights = compute_class_weight(class_weight='balanced', classes=[0,1], y=train_op)
def mortality_loss(y_true, y_pred):
    sample_weights = (1-y_true)*class_weights[0] + y_true*class_weights[1]
    bce = K.binary_crossentropy(y_true, y_pred)
    return K.mean(sample_weights*bce, axis=-1)
######################################################################################################## 
######################################################################################################## 

# var_weights = np.sum(fore_train_op[:, V:], axis=0)
# var_weights[var_weights==0] = var_weights.max()
# var_weights = var_weights.max()/var_weights
# var_weights = var_weights.reshape((1, V))
def forecast_loss(y_true, y_pred):
    return K.sum(y_true[:,V:]*(y_true[:,:V]-y_pred)**2, axis=-1)

def get_min_loss(weight):
    def min_loss(y_true, y_pred):
        return weight*y_pred
    return min_loss

class CustomCallback(Callback):
    def __init__(self, validation_data, batch_size):
        self.val_x, self.val_y = validation_data
        self.batch_size = batch_size
        super(Callback, self).__init__()

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.val_x, verbose=0, batch_size=self.batch_size)
        if type(y_pred)==type([]):
            y_pred = y_pred[0]
        precision, recall, thresholds = precision_recall_curve(self.val_y, y_pred)
        pr_auc = auc(recall, precision)
        roc_auc = roc_auc_score(self.val_y, y_pred)
        logs['custom_metric'] = pr_auc + roc_auc
        print ('val_aucs:', pr_auc, roc_auc)

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Embedding, Activation, Dropout, Softmax, Layer, InputSpec, Input, Dense, Lambda, TimeDistributed, Concatenate, Add
from tensorflow.keras import initializers, regularizers, constraints, Model
from tensorflow.python.keras.utils import tf_utils
from tensorflow.python.ops import array_ops
from tensorflow import nn

    
class CVE(Layer):
    def __init__(self, hid_units, output_dim):
        self.hid_units = hid_units
        self.output_dim = output_dim
        super(CVE, self).__init__()
        
    def build(self, input_shape): 
        self.W1 = self.add_weight(name='CVE_W1',
                            shape=(1, self.hid_units),
                            initializer='glorot_uniform',
                            trainable=True)
        self.b1 = self.add_weight(name='CVE_b1',
                            shape=(self.hid_units,),
                            initializer='zeros',
                            trainable=True)
        self.W2 = self.add_weight(name='CVE_W2',
                            shape=(self.hid_units, self.output_dim),
                            initializer='glorot_uniform',
                            trainable=True)
        super(CVE, self).build(input_shape)
        
    def call(self, x):
        x = K.expand_dims(x, axis=-1)
        x = K.dot(K.tanh(K.bias_add(K.dot(x, self.W1), self.b1)), self.W2)
        return x
        
    def compute_output_shape(self, input_shape):
        return input_shape + (self.output_dim,)
    
    
class Attention(Layer):
    
    def __init__(self, hid_dim):
        self.hid_dim = hid_dim
        super(Attention, self).__init__()

    def build(self, input_shape):
        d = input_shape.as_list()[-1]
        self.W = self.add_weight(shape=(d, self.hid_dim), name='Att_W',
                                 initializer='glorot_uniform',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.hid_dim,), name='Att_b',
                                 initializer='zeros',
                                 trainable=True)
        self.u = self.add_weight(shape=(self.hid_dim,1), name='Att_u',
                                 initializer='glorot_uniform',
                                 trainable=True)
        super(Attention, self).build(input_shape)
        
    def call(self, x, mask, mask_value=-1e30):
        attn_weights = K.dot(K.tanh(K.bias_add(K.dot(x,self.W), self.b)), self.u)
        mask = K.expand_dims(mask, axis=-1)
        attn_weights = mask*attn_weights + (1-mask)*mask_value
        attn_weights = K.softmax(attn_weights, axis=-2)
        return attn_weights
        
    def compute_output_shape(self, input_shape):
        return input_shape[:-1] + (1,)
    
    
class Transformer(Layer):
    
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0):
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = K.epsilon() * K.epsilon()
        super(Transformer, self).__init__()

    def build(self, input_shape):
        d = input_shape.as_list()[-1]
        if self.dk==None:
            self.dk = d//self.h
        if self.dv==None:
            self.dv = d//self.h
        if self.dff==None:
            self.dff = 2*d
        self.Wq = self.add_weight(shape=(self.N, self.h, d, self.dk), name='Wq',
                                 initializer='glorot_uniform', trainable=True)
        self.Wk = self.add_weight(shape=(self.N, self.h, d, self.dk), name='Wk',
                                 initializer='glorot_uniform', trainable=True)
        self.Wv = self.add_weight(shape=(self.N, self.h, d, self.dv), name='Wv',
                                 initializer='glorot_uniform', trainable=True)
        self.Wo = self.add_weight(shape=(self.N, self.dv*self.h, d), name='Wo',
                                 initializer='glorot_uniform', trainable=True)
        self.W1 = self.add_weight(shape=(self.N, d, self.dff), name='W1',
                                 initializer='glorot_uniform', trainable=True)
        self.b1 = self.add_weight(shape=(self.N, self.dff), name='b1',
                                 initializer='zeros', trainable=True)
        self.W2 = self.add_weight(shape=(self.N, self.dff, d), name='W2',
                                 initializer='glorot_uniform', trainable=True)
        self.b2 = self.add_weight(shape=(self.N, d), name='b2',
                                 initializer='zeros', trainable=True)
        self.gamma = self.add_weight(shape=(2*self.N,), name='gamma',
                                 initializer='ones', trainable=True)
        self.beta = self.add_weight(shape=(2*self.N,), name='beta',
                                 initializer='zeros', trainable=True)
        super(Transformer, self).build(input_shape)
        
    def call(self, x, mask, mask_value=-1e-30):
        mask = K.expand_dims(mask, axis=-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = K.dot(x, self.Wq[i,j,:,:])
                k = K.permute_dimensions(K.dot(x, self.Wk[i,j,:,:]), (0,2,1))
                v = K.dot(x, self.Wv[i,j,:,:])
                A = K.batch_dot(q,k)
                # Mask unobserved steps.
                A = mask*A + (1-mask)*mask_value
                # Mask for attention dropout.
                def dropped_A():
                    dp_mask = K.cast((K.random_uniform(shape=array_ops.shape(A))>=self.dropout), K.floatx())
                    return A*dp_mask + (1-dp_mask)*mask_value
                A = sc.smart_cond(K.learning_phase(), dropped_A, lambda: array_ops.identity(A))
                A = K.softmax(A, axis=-1)
                mha_ops.append(K.batch_dot(A,v))
            conc = K.concatenate(mha_ops, axis=-1)
            proj = K.dot(conc, self.Wo[i,:,:])
            # Dropout.
            proj = sc.smart_cond(K.learning_phase(), lambda: array_ops.identity(nn.dropout(proj, rate=self.dropout)),\
                                       lambda: array_ops.identity(proj))
            # Add & LN
            x = x+proj
            mean = K.mean(x, axis=-1, keepdims=True)
            variance = K.mean(K.square(x - mean), axis=-1, keepdims=True)
            std = K.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x*self.gamma[2*i] + self.beta[2*i]
            # FFN
            ffn_op = K.bias_add(K.dot(K.relu(K.bias_add(K.dot(x, self.W1[i,:,:]), self.b1[i,:])), 
                           self.W2[i,:,:]), self.b2[i,:,])
            # Dropout.
            ffn_op = sc.smart_cond(K.learning_phase(), lambda: array_ops.identity(nn.dropout(ffn_op, rate=self.dropout)),\
                                       lambda: array_ops.identity(ffn_op))
            # Add & LN
            x = x+ffn_op
            mean = K.mean(x, axis=-1, keepdims=True)
            variance = K.mean(K.square(x - mean), axis=-1, keepdims=True)
            std = K.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x*self.gamma[2*i+1] + self.beta[2*i+1]            
        return x
        
    def compute_output_shape(self, input_shape):
        return input_shape


def build_strats(D, max_len, V, d, N, he, dropout, forecast=False):
    demo = Input(shape=(D,))
    demo_enc = Dense(2*d, activation='tanh')(demo)
    demo_enc = Dense(d, activation='tanh')(demo_enc)
    varis = Input(shape=(max_len,))
    values = Input(shape=(max_len,))
    times = Input(shape=(max_len,))
    varis_emb = Embedding(V+1, d)(varis)
    cve_units = int(np.sqrt(d))
    values_emb = CVE(cve_units, d)(values)
    times_emb = CVE(cve_units, d)(times)
    comb_emb = Add()([varis_emb, values_emb, times_emb]) # b, L, d
#     demo_enc = Lambda(lambda x:K.expand_dims(x, axis=-2))(demo_enc) # b, 1, d
#     comb_emb = Concatenate(axis=-2)([demo_enc, comb_emb]) # b, L+1, d
    mask = Lambda(lambda x:K.clip(x,0,1))(varis) # b, L
#     mask = Lambda(lambda x:K.concatenate((K.ones_like(x)[:,0:1], x), axis=-1))(mask) # b, L+1
    cont_emb = Transformer(N, he, dk=None, dv=None, dff=None, dropout=dropout)(comb_emb, mask=mask)
    attn_weights = Attention(2*d)(cont_emb, mask=mask)
    fused_emb = Lambda(lambda x:K.sum(x[0]*x[1], axis=-2))([cont_emb, attn_weights])
    # embed text input
    texts = Input(shape=(33792,))
    text_enc = Dense(2*d, activation='tanh')(texts)
    text_enc = Dense(d, activation='tanh')(text_enc)
    conc = Concatenate(axis=-1)([fused_emb, text_enc, demo_enc])
    fore_op = Dense(V)(conc)
    op = Dense(1, activation='sigmoid')(fore_op)
    model = Model([demo, times, values, varis, texts], op)
    if forecast:
        fore_model = Model([demo, times, values, varis, texts], fore_op)
        return [model, fore_model]
    return model

# To tune:
# 1. Transformer parameters. (N, h, dropout)
# 2. Normalization

In [ ]:
repeats = {k:10 for k in [10,20,30,40,50,60]}
lds = [10]
batch_size, lr, patience = 32, 0.0005, 10
d, N, he, dropout = 50,2,4,0.2
fore_savepath = 'Exp0/models/forecasting/forecasting_50_epochs.h5'
f = open('log_text_10.csv', 'a+')
f.write('\nTraining on different % of labeled data\n')

train_inds = np.arange(len(train_op))
valid_inds = np.arange(len(valid_op))
gen_res = {}

np.random.seed(2021)
for ld in lds:
    np.random.shuffle(train_inds)
    np.random.shuffle(valid_inds)
    train_starts = [int(i) for i in np.linspace(0, len(train_inds)-int(ld*len(train_inds)/100), repeats[ld])]
    valid_starts = [int(i) for i in np.linspace(0, len(valid_inds)-int(ld*len(valid_inds)/100), repeats[ld])]
    f.write('Training on '+str(ld)+' % of labaled data+\n'+'val_metric,roc_auc,pr_auc,min_rp,savepath\n')
    all_test_res = []
    for i in range(repeats[ld]):
        print ('Repeat', i, 'ld', ld)
        # Get train and validation data.
        curr_train_ind = train_inds[np.arange(train_starts[i], train_starts[i]+int(ld*len(train_inds)/100))]
        curr_valid_ind = valid_inds[np.arange(valid_starts[i], valid_starts[i]+int(ld*len(valid_inds)/100))]
        curr_train_ip = [ip[curr_train_ind] for ip in train_ip]
        curr_valid_ip = [ip[curr_valid_ind] for ip in valid_ip]
        curr_train_op = train_op[curr_train_ind]
        curr_valid_op = valid_op[curr_valid_ind]
        print ('Num train:',len(curr_train_op),'Num valid:',len(curr_valid_op))
        # Construct save_path.
        savepath = 'new_mimic_iii_24hm_strats_no_interp_with_ss_repeat'+str(i)+'_'+str(ld)+'ld'+'.h5'
        print (savepath)
        # Build and compile model.
        model, fore_model =  build_strats(D, max_len, V, d, N, he, dropout, forecast=True)
        model.compile(loss=mortality_loss, optimizer=Adam(lr))
        fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
        # Load pretrained weights here.
        fore_model.load_weights(fore_savepath)
        # Train model.
        es = EarlyStopping(monitor='custom_metric', patience=patience, mode='max', 
                           restore_best_weights=True)
        cus = CustomCallback(validation_data=(curr_valid_ip, curr_valid_op), batch_size=batch_size)
        his = model.fit(curr_train_ip, curr_train_op, batch_size=batch_size, epochs=1000,
                        verbose=1, callbacks=[cus, es]).history
        model.save_weights(savepath)
        # Test and write to log.
        rocauc, prauc, minrp = get_res(test_op, model.predict(test_ip, verbose=0, batch_size=batch_size))
        f.write(str(np.min(his['custom_metric']))+str(rocauc)+str(prauc)+str(minrp)+savepath+'\n')
        print ('Test res', rocauc, prauc, minrp)
        all_test_res.append([rocauc, prauc, minrp])
        
    gen_res[ld] = []
    for i in range(len(all_test_res[0])):
        nums = [test_res[i] for test_res in all_test_res]
        gen_res[ld].append((np.mean(nums), np.std(nums)))
    print ('gen_res', gen_res)
f.close()

# # save to local
# log_path = '/content/log.csv'
# files.download(log_path)

In [ ]:
# repeats = {k:10 for k in [10,20,30,40,50,60]}
# lds = [20]
# batch_size, lr, patience = 32, 0.0005, 10
# d, N, he, dropout = 50,2,4,0.2
# fore_savepath = 'Exp0/models/forecasting/forecasting_50_epochs.h5'
# f = open('log_text_20.csv', 'a+')
# f.write('\nTraining on different % of labeled data\n')

# train_inds = np.arange(len(train_op))
# valid_inds = np.arange(len(valid_op))
# gen_res = {}

# np.random.seed(2021)
# for ld in lds:
#     np.random.shuffle(train_inds)
#     np.random.shuffle(valid_inds)
#     train_starts = [int(i) for i in np.linspace(0, len(train_inds)-int(ld*len(train_inds)/100), repeats[ld])]
#     valid_starts = [int(i) for i in np.linspace(0, len(valid_inds)-int(ld*len(valid_inds)/100), repeats[ld])]
#     f.write('Training on '+str(ld)+' % of labaled data+\n'+'val_metric,roc_auc,pr_auc,min_rp,savepath\n')
#     all_test_res = []
#     for i in range(repeats[ld]):
#         print ('Repeat', i, 'ld', ld)
#         # Get train and validation data.
#         curr_train_ind = train_inds[np.arange(train_starts[i], train_starts[i]+int(ld*len(train_inds)/100))]
#         curr_valid_ind = valid_inds[np.arange(valid_starts[i], valid_starts[i]+int(ld*len(valid_inds)/100))]
#         curr_train_ip = [ip[curr_train_ind] for ip in train_ip]
#         curr_valid_ip = [ip[curr_valid_ind] for ip in valid_ip]
#         curr_train_op = train_op[curr_train_ind]
#         curr_valid_op = valid_op[curr_valid_ind]
#         print ('Num train:',len(curr_train_op),'Num valid:',len(curr_valid_op))
#         # Construct save_path.
#         savepath = 'new_mimic_iii_24hm_strats_no_interp_with_ss_repeat'+str(i)+'_'+str(ld)+'ld'+'.h5'
#         print (savepath)
#         # Build and compile model.
#         model, fore_model =  build_strats(D, max_len, V, d, N, he, dropout, forecast=True)
#         model.compile(loss=mortality_loss, optimizer=Adam(lr))
#         fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
#         # Load pretrained weights here.
#         fore_model.load_weights(fore_savepath)
#         # Train model.
#         es = EarlyStopping(monitor='custom_metric', patience=patience, mode='max', 
#                            restore_best_weights=True)
#         cus = CustomCallback(validation_data=(curr_valid_ip, curr_valid_op), batch_size=batch_size)
#         his = model.fit(curr_train_ip, curr_train_op, batch_size=batch_size, epochs=1000,
#                         verbose=1, callbacks=[cus, es]).history
#         model.save_weights(savepath)
#         # Test and write to log.
#         rocauc, prauc, minrp = get_res(test_op, model.predict(test_ip, verbose=0, batch_size=batch_size))
#         f.write(str(np.min(his['custom_metric']))+str(rocauc)+str(prauc)+str(minrp)+savepath+'\n')
#         print ('Test res', rocauc, prauc, minrp)
#         all_test_res.append([rocauc, prauc, minrp])
        
#     gen_res[ld] = []
#     for i in range(len(all_test_res[0])):
#         nums = [test_res[i] for test_res in all_test_res]
#         gen_res[ld].append((np.mean(nums), np.std(nums)))
#     print ('gen_res', gen_res)
# f.close()

# # # save to local
# # log_path = '/content/log.csv'
# # files.download(log_path)

In [ ]:
# repeats = {k:10 for k in [10,20,30,40,50,60]}
# lds = [30]
# batch_size, lr, patience = 32, 0.0005, 10
# d, N, he, dropout = 50,2,4,0.2
# fore_savepath = 'Exp0/models/forecasting/forecasting_50_epochs.h5'
# f = open('log_text_30.csv', 'a+')
# f.write('\nTraining on different % of labeled data\n')

# train_inds = np.arange(len(train_op))
# valid_inds = np.arange(len(valid_op))
# gen_res = {}

# np.random.seed(2021)
# for ld in lds:
#     np.random.shuffle(train_inds)
#     np.random.shuffle(valid_inds)
#     train_starts = [int(i) for i in np.linspace(0, len(train_inds)-int(ld*len(train_inds)/100), repeats[ld])]
#     valid_starts = [int(i) for i in np.linspace(0, len(valid_inds)-int(ld*len(valid_inds)/100), repeats[ld])]
#     f.write('Training on '+str(ld)+' % of labaled data+\n'+'val_metric,roc_auc,pr_auc,min_rp,savepath\n')
#     all_test_res = []
#     for i in range(repeats[ld]):
#         print ('Repeat', i, 'ld', ld)
#         # Get train and validation data.
#         curr_train_ind = train_inds[np.arange(train_starts[i], train_starts[i]+int(ld*len(train_inds)/100))]
#         curr_valid_ind = valid_inds[np.arange(valid_starts[i], valid_starts[i]+int(ld*len(valid_inds)/100))]
#         curr_train_ip = [ip[curr_train_ind] for ip in train_ip]
#         curr_valid_ip = [ip[curr_valid_ind] for ip in valid_ip]
#         curr_train_op = train_op[curr_train_ind]
#         curr_valid_op = valid_op[curr_valid_ind]
#         print ('Num train:',len(curr_train_op),'Num valid:',len(curr_valid_op))
#         # Construct save_path.
#         savepath = 'new_mimic_iii_24hm_strats_no_interp_with_ss_repeat'+str(i)+'_'+str(ld)+'ld'+'.h5'
#         print (savepath)
#         # Build and compile model.
#         model, fore_model =  build_strats(D, max_len, V, d, N, he, dropout, forecast=True)
#         model.compile(loss=mortality_loss, optimizer=Adam(lr))
#         fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
#         # Load pretrained weights here.
#         fore_model.load_weights(fore_savepath)
#         # Train model.
#         es = EarlyStopping(monitor='custom_metric', patience=patience, mode='max', 
#                            restore_best_weights=True)
#         cus = CustomCallback(validation_data=(curr_valid_ip, curr_valid_op), batch_size=batch_size)
#         his = model.fit(curr_train_ip, curr_train_op, batch_size=batch_size, epochs=1000,
#                         verbose=1, callbacks=[cus, es]).history
#         model.save_weights(savepath)
#         # Test and write to log.
#         rocauc, prauc, minrp = get_res(test_op, model.predict(test_ip, verbose=0, batch_size=batch_size))
#         f.write(str(np.min(his['custom_metric']))+str(rocauc)+str(prauc)+str(minrp)+savepath+'\n')
#         print ('Test res', rocauc, prauc, minrp)
#         all_test_res.append([rocauc, prauc, minrp])
        
#     gen_res[ld] = []
#     for i in range(len(all_test_res[0])):
#         nums = [test_res[i] for test_res in all_test_res]
#         gen_res[ld].append((np.mean(nums), np.std(nums)))
#     print ('gen_res', gen_res)
# f.close()

# # # save to local
# # log_path = '/content/log.csv'
# # files.download(log_path)

In [ ]:
repeats = {k:10 for k in [10,20,30,40,50,60]}
lds = [50]
batch_size, lr, patience = 32, 0.0005, 10
d, N, he, dropout = 50,2,4,0.2
fore_savepath = 'Exp0/models/forecasting/forecasting_50_epochs.h5'
f = open('Exp0/logs/log_text_50.csv', 'a+')
f.write('\nTraining on different % of labeled data\n')

train_inds = np.arange(len(train_op))
valid_inds = np.arange(len(valid_op))
gen_res = {}

np.random.seed(2021)
for ld in lds:
    np.random.shuffle(train_inds)
    np.random.shuffle(valid_inds)
    train_starts = [int(i) for i in np.linspace(0, len(train_inds)-int(ld*len(train_inds)/100), repeats[ld])]
    valid_starts = [int(i) for i in np.linspace(0, len(valid_inds)-int(ld*len(valid_inds)/100), repeats[ld])]
    f.write('Training on '+str(ld)+' % of labaled data+\n'+'val_metric,roc_auc,pr_auc,min_rp,savepath\n')
    all_test_res = []
    for i in range(repeats[ld]):
        print ('Repeat', i, 'ld', ld)
        # Get train and validation data.
        curr_train_ind = train_inds[np.arange(train_starts[i], train_starts[i]+int(ld*len(train_inds)/100))]
        curr_valid_ind = valid_inds[np.arange(valid_starts[i], valid_starts[i]+int(ld*len(valid_inds)/100))]
        curr_train_ip = [ip[curr_train_ind] for ip in train_ip]
        curr_valid_ip = [ip[curr_valid_ind] for ip in valid_ip]
        curr_train_op = train_op[curr_train_ind]
        curr_valid_op = valid_op[curr_valid_ind]
        print ('Num train:',len(curr_train_op),'Num valid:',len(curr_valid_op))
        # Construct save_path.
        savepath = 'Exp0/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat'+str(i)+'_'+str(ld)+'ld'+'.h5'
        print (savepath)
        # Build and compile model.
        model, fore_model =  build_strats(D, max_len, V, d, N, he, dropout, forecast=True)
        model.compile(loss=mortality_loss, optimizer=Adam(lr))
        fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
        # Load pretrained weights here.
        fore_model.load_weights(fore_savepath)
        # Train model.
        es = EarlyStopping(monitor='custom_metric', patience=patience, mode='max', 
                           restore_best_weights=True)
        cus = CustomCallback(validation_data=(curr_valid_ip, curr_valid_op), batch_size=batch_size)
        his = model.fit(curr_train_ip, curr_train_op, batch_size=batch_size, epochs=1000,
                        verbose=1, callbacks=[cus, es]).history
        model.save_weights(savepath)
        # Test and write to log.
        rocauc, prauc, minrp = get_res(test_op, model.predict(test_ip, verbose=0, batch_size=batch_size))
        f.write(str(np.min(his['custom_metric']))+str(rocauc)+str(prauc)+str(minrp)+savepath+'\n')
        print ('Test res', rocauc, prauc, minrp)
        all_test_res.append([rocauc, prauc, minrp])
        
    gen_res[ld] = []
    for i in range(len(all_test_res[0])):
        nums = [test_res[i] for test_res in all_test_res]
        gen_res[ld].append((np.mean(nums), np.std(nums)))
    print ('gen_res', gen_res)
f.close()

# # save to local
# log_path = '/content/log.csv'
# files.download(log_path)

Repeat 0 ld 50
Num train: 18275 Num valid: 4631
Exp0/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat0_50ld.h5


2023-07-09 17:28:17.547859: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30503 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3a:00.0, compute capability: 7.0


Epoch 1/1000


2023-07-09 17:28:52.196962: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x562e7dac1a20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-09 17:28:52.196997: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2023-07-09 17:28:52.209720: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-09 17:28:52.488855: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2023-07-09 17:28:53.027911: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


572/572 [==============================] - 53s 55ms/step - loss: 0.4740 - custom_metric: 1.4201
Epoch 2/1000
572/572 [==============================] - 28s 48ms/step - loss: 0.4139 - custom_metric: 1.4379
Epoch 3/1000
572/572 [==============================] - 27s 48ms/step - loss: 0.3856 - custom_metric: 1.4465
Epoch 4/1000
572/572 [==============================] - 27s 48ms/step - loss: 0.3655 - custom_metric: 1.4258
Epoch 5/1000
572/572 [==============================] - 27s 47ms/step - loss: 0.3403 - custom_metric: 1.4099
Epoch 6/1000
572/572 [==============================] - 27s 47ms/step - loss: 0.3118 - custom_metric: 1.3965
Epoch 7/1000
572/572 [==============================] - 27s 47ms/step - loss: 0.2858 - custom_metric: 1.3757
Epoch 8/1000
572/572 [==============================] - 27s 47ms/step - loss: 0.2579 - custom_metric: 1.3984
Epoch 9/1000
572/572 [==============================] - 27s 47ms/step - loss: 0.2302 - custom_metric: 1.3321
Epoch 10/1000
572/572 [=========

In [43]:
repeats = {k:10 for k in [10,20,30,40,50,60]}
lds = [40]
batch_size, lr, patience = 32, 0.0005, 10
d, N, he, dropout = 50,2,4,0.2
fore_savepath = 'Exp0/models/forecasting/forecasting_50_epochs.h5'
f = open('Exp0/logs/log_text_90.csv', 'a+')
f.write('\nTraining on different % of labeled data\n')

train_inds = np.arange(len(train_op))
valid_inds = np.arange(len(valid_op))
gen_res = {}

np.random.seed(2021)
for ld in lds:
    np.random.shuffle(train_inds)
    np.random.shuffle(valid_inds)
    train_starts = [int(i) for i in np.linspace(0, len(train_inds)-int(ld*len(train_inds)/100), repeats[ld])]
    valid_starts = [int(i) for i in np.linspace(0, len(valid_inds)-int(ld*len(valid_inds)/100), repeats[ld])]
    f.write('Training on '+str(ld)+' % of labaled data+\n'+'val_metric,roc_auc,pr_auc,min_rp,savepath\n')
    all_test_res = []
    for i in range(repeats[ld]):
        print ('Repeat', i, 'ld', ld)
        # Get train and validation data.
        curr_train_ind = train_inds[np.arange(train_starts[i], train_starts[i]+int(ld*len(train_inds)/100))]
        curr_valid_ind = valid_inds[np.arange(valid_starts[i], valid_starts[i]+int(ld*len(valid_inds)/100))]
        curr_train_ip = [ip[curr_train_ind] for ip in train_ip]
        curr_valid_ip = [ip[curr_valid_ind] for ip in valid_ip]
        curr_train_op = train_op[curr_train_ind]
        curr_valid_op = valid_op[curr_valid_ind]
        print ('Num train:',len(curr_train_op),'Num valid:',len(curr_valid_op))
        # Construct save_path.
        savepath = 'Exp0/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat'+str(i)+'_'+str(ld)+'ld'+'.h5'
        print (savepath)
        # Build and compile model.
        model, fore_model =  build_strats(D, max_len, V, d, N, he, dropout, forecast=True)
        model.compile(loss=mortality_loss, optimizer=Adam(lr))
        fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
        # Load pretrained weights here.
        fore_model.load_weights(fore_savepath)
        # Train model.
        es = EarlyStopping(monitor='custom_metric', patience=patience, mode='max', 
                           restore_best_weights=True)
        cus = CustomCallback(validation_data=(curr_valid_ip, curr_valid_op), batch_size=batch_size)
        his = model.fit(curr_train_ip, curr_train_op, batch_size=batch_size, epochs=1000,
                        verbose=1, callbacks=[cus, es]).history
        model.save_weights(savepath)
        # Test and write to log.
        rocauc, prauc, minrp = get_res(test_op, model.predict(test_ip, verbose=0, batch_size=batch_size))
        f.write(str(np.min(his['custom_metric']))+str(rocauc)+str(prauc)+str(minrp)+savepath+'\n')
        print ('Test res', rocauc, prauc, minrp)
        all_test_res.append([rocauc, prauc, minrp])
        
    gen_res[ld] = []
    for i in range(len(all_test_res[0])):
        nums = [test_res[i] for test_res in all_test_res]
        gen_res[ld].append((np.mean(nums), np.std(nums)))
    print ('gen_res', gen_res)
f.close()

# # save to local
# log_path = '/content/log.csv'
# files.download(log_path)

Repeat 0 ld 40
Num train: 14620 Num valid: 3704
Exp0/models/classification/new_mimic_iii_24hm_strats_no_interp_with_ss_repeat0_40ld.h5
Epoch 1/1000
457/457 [==============================] - 35s 55ms/step - loss: 0.4942 - custom_metric: 1.3584
Epoch 2/1000
457/457 [==============================] - 22s 47ms/step - loss: 0.4266 - custom_metric: 1.3610
Epoch 3/1000
457/457 [==============================] - 22s 48ms/step - loss: 0.3981 - custom_metric: 1.4075
Epoch 4/1000
457/457 [==============================] - 22s 47ms/step - loss: 0.3691 - custom_metric: 1.3583
Epoch 5/1000
457/457 [==============================] - 22s 47ms/step - loss: 0.3440 - custom_metric: 1.3838
Epoch 6/1000
457/457 [==============================] - 22s 47ms/step - loss: 0.3132 - custom_metric: 1.3454
Epoch 7/1000
457/457 [==============================] - 22s 47ms/step - loss: 0.2898 - custom_metric: 1.3657
Epoch 8/1000
457/457 [==============================] - 22s 47ms/step - loss: 0.2530 - custom_metric: 

In [ ]:
# repeats = {k:10 for k in [10,20,30,40,50,60]}
# lds = [40]
# batch_size, lr, patience = 32, 0.0005, 10
# d, N, he, dropout = 50,2,4,0.2
# fore_savepath = 'mimic_iii_24h_strats_no_interp_with_ss_fore_text_5.h5'
# f = open('log_text_40.csv', 'a+')
# f.write('\nTraining on different % of labeled data\n')

# train_inds = np.arange(len(train_op))
# valid_inds = np.arange(len(valid_op))
# gen_res = {}

# np.random.seed(2021)
# for ld in lds:
#     np.random.shuffle(train_inds)
#     np.random.shuffle(valid_inds)
#     train_starts = [int(i) for i in np.linspace(0, len(train_inds)-int(ld*len(train_inds)/100), repeats[ld])]
#     valid_starts = [int(i) for i in np.linspace(0, len(valid_inds)-int(ld*len(valid_inds)/100), repeats[ld])]
#     f.write('Training on '+str(ld)+' % of labaled data+\n'+'val_metric,roc_auc,pr_auc,min_rp,savepath\n')
#     all_test_res = []
#     for i in range(repeats[ld]):
#         print ('Repeat', i, 'ld', ld)
#         # Get train and validation data.
#         curr_train_ind = train_inds[np.arange(train_starts[i], train_starts[i]+int(ld*len(train_inds)/100))]
#         curr_valid_ind = valid_inds[np.arange(valid_starts[i], valid_starts[i]+int(ld*len(valid_inds)/100))]
#         curr_train_ip = [ip[curr_train_ind] for ip in train_ip]
#         curr_valid_ip = [ip[curr_valid_ind] for ip in valid_ip]
#         curr_train_op = train_op[curr_train_ind]
#         curr_valid_op = valid_op[curr_valid_ind]
#         print ('Num train:',len(curr_train_op),'Num valid:',len(curr_valid_op))
#         # Construct save_path.
#         savepath = 'new_mimic_iii_24hm_strats_no_interp_with_ss_repeat'+str(i)+'_'+str(ld)+'ld'+'.h5'
#         print (savepath)
#         # Build and compile model.
#         model, fore_model =  build_strats(D, max_len, V, d, N, he, dropout, forecast=True)
#         model.compile(loss=mortality_loss, optimizer=Adam(lr))
#         fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
#         # Load pretrained weights here.
#         fore_model.load_weights(fore_savepath)
#         # Train model.
#         es = EarlyStopping(monitor='custom_metric', patience=patience, mode='max', 
#                            restore_best_weights=True)
#         cus = CustomCallback(validation_data=(curr_valid_ip, curr_valid_op), batch_size=batch_size)
#         his = model.fit(curr_train_ip, curr_train_op, batch_size=batch_size, epochs=1000,
#                         verbose=1, callbacks=[cus, es]).history
#         model.save_weights(savepath)
#         # Test and write to log.
#         rocauc, prauc, minrp = get_res(test_op, model.predict(test_ip, verbose=0, batch_size=batch_size))
#         f.write(str(np.min(his['custom_metric']))+str(rocauc)+str(prauc)+str(minrp)+savepath+'\n')
#         print ('Test res', rocauc, prauc, minrp)
#         all_test_res.append([rocauc, prauc, minrp])
        
#     gen_res[ld] = []
#     for i in range(len(all_test_res[0])):
#         nums = [test_res[i] for test_res in all_test_res]
#         gen_res[ld].append((np.mean(nums), np.std(nums)))
#     print ('gen_res', gen_res)
# f.close()

# # # save to local
# # log_path = '/content/log.csv'
# # files.download(log_path)

In [ ]:
# repeats = {k:10 for k in [10,20,30,40,50,60]}
# lds = [50]
# batch_size, lr, patience = 32, 0.0005, 10
# d, N, he, dropout = 50,2,4,0.2
# fore_savepath = 'mimic_iii_24h_strats_no_interp_with_ss_fore_text_5.h5'
# f = open('log_text_50.csv', 'a+')
# f.write('\nTraining on different % of labeled data\n')

# train_inds = np.arange(len(train_op))
# valid_inds = np.arange(len(valid_op))
# gen_res = {}

# np.random.seed(2021)
# for ld in lds:
#     np.random.shuffle(train_inds)
#     np.random.shuffle(valid_inds)
#     train_starts = [int(i) for i in np.linspace(0, len(train_inds)-int(ld*len(train_inds)/100), repeats[ld])]
#     valid_starts = [int(i) for i in np.linspace(0, len(valid_inds)-int(ld*len(valid_inds)/100), repeats[ld])]
#     f.write('Training on '+str(ld)+' % of labaled data+\n'+'val_metric,roc_auc,pr_auc,min_rp,savepath\n')
#     all_test_res = []
#     for i in range(repeats[ld]):
#         print ('Repeat', i, 'ld', ld)
#         # Get train and validation data.
#         curr_train_ind = train_inds[np.arange(train_starts[i], train_starts[i]+int(ld*len(train_inds)/100))]
#         curr_valid_ind = valid_inds[np.arange(valid_starts[i], valid_starts[i]+int(ld*len(valid_inds)/100))]
#         curr_train_ip = [ip[curr_train_ind] for ip in train_ip]
#         curr_valid_ip = [ip[curr_valid_ind] for ip in valid_ip]
#         curr_train_op = train_op[curr_train_ind]
#         curr_valid_op = valid_op[curr_valid_ind]
#         print ('Num train:',len(curr_train_op),'Num valid:',len(curr_valid_op))
#         # Construct save_path.
#         savepath = 'new_mimic_iii_24hm_strats_no_interp_with_ss_repeat'+str(i)+'_'+str(ld)+'ld'+'.h5'
#         print (savepath)
#         # Build and compile model.
#         model, fore_model =  build_strats(D, max_len, V, d, N, he, dropout, forecast=True)
#         model.compile(loss=mortality_loss, optimizer=Adam(lr))
#         fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
#         # Load pretrained weights here.
#         fore_model.load_weights(fore_savepath)
#         # Train model.
#         es = EarlyStopping(monitor='custom_metric', patience=patience, mode='max', 
#                            restore_best_weights=True)
#         cus = CustomCallback(validation_data=(curr_valid_ip, curr_valid_op), batch_size=batch_size)
#         his = model.fit(curr_train_ip, curr_train_op, batch_size=batch_size, epochs=1000,
#                         verbose=1, callbacks=[cus, es]).history
#         model.save_weights(savepath)
#         # Test and write to log.
#         rocauc, prauc, minrp = get_res(test_op, model.predict(test_ip, verbose=0, batch_size=batch_size))
#         f.write(str(np.min(his['custom_metric']))+str(rocauc)+str(prauc)+str(minrp)+savepath+'\n')
#         print ('Test res', rocauc, prauc, minrp)
#         all_test_res.append([rocauc, prauc, minrp])
        
#     gen_res[ld] = []
#     for i in range(len(all_test_res[0])):
#         nums = [test_res[i] for test_res in all_test_res]
#         gen_res[ld].append((np.mean(nums), np.std(nums)))
#     print ('gen_res', gen_res)
# f.close()

# # # save to local
# # log_path = '/content/log.csv'
# # files.download(log_path)

In [ ]:
# repeats = {k:10 for k in [10,20,30,40,50,60]}
# lds = [90]
# batch_size, lr, patience = 32, 0.0005, 10
# d, N, he, dropout = 50,2,4,0.2
# fore_savepath = 'mimic_iii_24h_strats_no_interp_with_ss_fore_text_5.h5'
# f = open('log_text_90.csv', 'a+')
# f.write('\nTraining on different % of labeled data\n')

# train_inds = np.arange(len(train_op))
# valid_inds = np.arange(len(valid_op))
# gen_res = {}

# np.random.seed(2021)
# for ld in lds:
#     np.random.shuffle(train_inds)
#     np.random.shuffle(valid_inds)
#     train_starts = [int(i) for i in np.linspace(0, len(train_inds)-int(ld*len(train_inds)/100), repeats[ld])]
#     valid_starts = [int(i) for i in np.linspace(0, len(valid_inds)-int(ld*len(valid_inds)/100), repeats[ld])]
#     f.write('Training on '+str(ld)+' % of labaled data+\n'+'val_metric,roc_auc,pr_auc,min_rp,savepath\n')
#     all_test_res = []
#     for i in range(repeats[ld]):
#         print ('Repeat', i, 'ld', ld)
#         # Get train and validation data.
#         curr_train_ind = train_inds[np.arange(train_starts[i], train_starts[i]+int(ld*len(train_inds)/100))]
#         curr_valid_ind = valid_inds[np.arange(valid_starts[i], valid_starts[i]+int(ld*len(valid_inds)/100))]
#         curr_train_ip = [ip[curr_train_ind] for ip in train_ip]
#         curr_valid_ip = [ip[curr_valid_ind] for ip in valid_ip]
#         curr_train_op = train_op[curr_train_ind]
#         curr_valid_op = valid_op[curr_valid_ind]
#         print ('Num train:',len(curr_train_op),'Num valid:',len(curr_valid_op))
#         # Construct save_path.
#         savepath = 'new_mimic_iii_24hm_strats_no_interp_with_ss_repeat'+str(i)+'_'+str(ld)+'ld'+'.h5'
#         print (savepath)
#         # Build and compile model.
#         model, fore_model =  build_strats(D, max_len, V, d, N, he, dropout, forecast=True)
#         model.compile(loss=mortality_loss, optimizer=Adam(lr))
#         fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
#         # Load pretrained weights here.
#         fore_model.load_weights(fore_savepath)
#         # Train model.
#         es = EarlyStopping(monitor='custom_metric', patience=patience, mode='max', 
#                            restore_best_weights=True)
#         cus = CustomCallback(validation_data=(curr_valid_ip, curr_valid_op), batch_size=batch_size)
#         his = model.fit(curr_train_ip, curr_train_op, batch_size=batch_size, epochs=1000,
#                         verbose=1, callbacks=[cus, es]).history
#         model.save_weights(savepath)
#         # Test and write to log.
#         rocauc, prauc, minrp = get_res(test_op, model.predict(test_ip, verbose=0, batch_size=batch_size))
#         f.write(str(np.min(his['custom_metric']))+str(rocauc)+str(prauc)+str(minrp)+savepath+'\n')
#         print ('Test res', rocauc, prauc, minrp)
#         all_test_res.append([rocauc, prauc, minrp])
        
#     gen_res[ld] = []
#     for i in range(len(all_test_res[0])):
#         nums = [test_res[i] for test_res in all_test_res]
#         gen_res[ld].append((np.mean(nums), np.std(nums)))
#     print ('gen_res', gen_res)
# f.close()

# # # save to local
# # log_path = '/content/log.csv'
# # files.download(log_path)

In [ ]:
# repeats = {k:10 for k in [10,20,30,40,50,60]}
# lds = [100]
# batch_size, lr, patience = 32, 0.0005, 10
# d, N, he, dropout = 50,2,4,0.2
# fore_savepath = 'mimic_iii_24h_strats_no_interp_with_ss_fore_text_5.h5'
# f = open('log_text_100.csv', 'a+')
# f.write('\nTraining on different % of labeled data\n')

# train_inds = np.arange(len(train_op))
# valid_inds = np.arange(len(valid_op))
# gen_res = {}

# np.random.seed(2021)
# for ld in lds:
#     np.random.shuffle(train_inds)
#     np.random.shuffle(valid_inds)
#     train_starts = [int(i) for i in np.linspace(0, len(train_inds)-int(ld*len(train_inds)/100), repeats[ld])]
#     valid_starts = [int(i) for i in np.linspace(0, len(valid_inds)-int(ld*len(valid_inds)/100), repeats[ld])]
#     f.write('Training on '+str(ld)+' % of labaled data+\n'+'val_metric,roc_auc,pr_auc,min_rp,savepath\n')
#     all_test_res = []
#     for i in range(repeats[ld]):
#         print ('Repeat', i, 'ld', ld)
#         # Get train and validation data.
#         curr_train_ind = train_inds[np.arange(train_starts[i], train_starts[i]+int(ld*len(train_inds)/100))]
#         curr_valid_ind = valid_inds[np.arange(valid_starts[i], valid_starts[i]+int(ld*len(valid_inds)/100))]
#         curr_train_ip = [ip[curr_train_ind] for ip in train_ip]
#         curr_valid_ip = [ip[curr_valid_ind] for ip in valid_ip]
#         curr_train_op = train_op[curr_train_ind]
#         curr_valid_op = valid_op[curr_valid_ind]
#         print ('Num train:',len(curr_train_op),'Num valid:',len(curr_valid_op))
#         # Construct save_path.
#         savepath = 'new_mimic_iii_24hm_strats_no_interp_with_ss_repeat'+str(i)+'_'+str(ld)+'ld'+'.h5'
#         print (savepath)
#         # Build and compile model.
#         model, fore_model =  build_strats(D, max_len, V, d, N, he, dropout, forecast=True)
#         model.compile(loss=mortality_loss, optimizer=Adam(lr))
#         fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
#         # Load pretrained weights here.
#         fore_model.load_weights(fore_savepath)
#         # Train model.
#         es = EarlyStopping(monitor='custom_metric', patience=patience, mode='max', 
#                            restore_best_weights=True)
#         cus = CustomCallback(validation_data=(curr_valid_ip, curr_valid_op), batch_size=batch_size)
#         his = model.fit(curr_train_ip, curr_train_op, batch_size=batch_size, epochs=1000,
#                         verbose=1, callbacks=[cus, es]).history
#         model.save_weights(savepath)
#         # Test and write to log.
#         rocauc, prauc, minrp = get_res(test_op, model.predict(test_ip, verbose=0, batch_size=batch_size))
#         f.write(str(np.min(his['custom_metric']))+str(rocauc)+str(prauc)+str(minrp)+savepath+'\n')
#         print ('Test res', rocauc, prauc, minrp)
#         all_test_res.append([rocauc, prauc, minrp])
        
#     gen_res[ld] = []
#     for i in range(len(all_test_res[0])):
#         nums = [test_res[i] for test_res in all_test_res]
#         gen_res[ld].append((np.mean(nums), np.std(nums)))
#     print ('gen_res', gen_res)
# f.close()

# # # save to local
# # log_path = '/content/log.csv'
# # files.download(log_path)